In [1]:
import numpy as np
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, TimeDistributed
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from keras.layers import Input, Dense
from keras.models import Model
%matplotlib inline
import pandas as pd
from keras.models import Sequential

from utils import *

Using TensorFlow backend.


#### Get data

In [67]:
# dataset
dataset='ADA'

fpaths = []
labels = []
boy_fpaths = []
boy_labels = []

word_spoken = []

dataset = '41{}'.format(dataset)
input_folder = '..\data\{}'.format(dataset)

# pars the input directory that contains audio files
# get audio files and their lables

for f in os.listdir(input_folder):
    for w in os.listdir(input_folder+'\\'+ f):
        # check wheter files is wav or not
        
        
        
        if (w.find('wav')!=-1 and w.find('closed')==-1):
        
            idx_schunk=w.find('schunk')
            idx_chunk=w.find('chunk')
            
            if (idx_chunk!=idx_schunk==-1):
                if (int(w[idx_chunk-2:idx_chunk])<80):
                    fpaths.append(input_folder+'\\'+f+'\\'+w)
                    labels.append(f)
                    
                else:
                    boy_fpaths.append(input_folder+'\\'+f+'\\'+w)
                    boy_labels.append(f)
                
            else:
                boy_fpaths.append(input_folder+'\\'+f+'\\'+w)
                boy_labels.append(f)

                
            if f not in word_spoken:
                word_spoken.append(f)
print("Spoken words: "+ str(word_spoken))

Spoken words: ['0_zero', '1_one', '2_two', '3_three', '4_four', '5_five', '6_six', '7_seven', '8_eight', '9_nine', 'aboard', 'adjusted & locked', 'All switches', 'Alternate air door', 'A_C Documents', 'Battery+Main bus', 'Cabin doors', 'Checked', 'Circuit breakers', 'Cockpit', 'Cockpit cheklist completed', 'Completed', 'Decimal', 'Flight Controls', 'Fuel Quantity', 'Fuel Selector', 'Fuel Shutoff Valve', 'Fuel Temperature', 'in', 'locked', 'off', 'On', 'open', 'preflight inspection', 'removed', 'Seats & Belts', 'Shut-off cabin heat', 'sufficient', 'Towbar', 'Weight and balance']


In [68]:
# size of dataset
print(len(labels))

7108


In [69]:
print(len(boy_labels))

1969


In [70]:
# girls
label_set=list(set(labels))
label_set.sort()

# boys 
boy_label_set=list(set(boy_labels))
boy_label_set.sort()

In [71]:
label_set

['0_zero',
 '1_one',
 '2_two',
 '3_three',
 '4_four',
 '5_five',
 '6_six',
 '7_seven',
 '8_eight',
 '9_nine',
 'A_C Documents',
 'All switches',
 'Alternate air door',
 'Battery+Main bus',
 'Cabin doors',
 'Checked',
 'Circuit breakers',
 'Cockpit',
 'Cockpit cheklist completed',
 'Completed',
 'Decimal',
 'Flight Controls',
 'Fuel Quantity',
 'Fuel Selector',
 'Fuel Shutoff Valve',
 'Fuel Temperature',
 'On',
 'Seats & Belts',
 'Shut-off cabin heat',
 'Towbar',
 'Weight and balance',
 'aboard',
 'adjusted & locked',
 'in',
 'locked',
 'off',
 'open',
 'preflight inspection',
 'removed',
 'sufficient']

## Extracting frequeny domain features

At the second stage we convet a signal into the freqency domain. In monst modern speech recognitoon freqeency-domain features are used as key component. In case of multispeakers MFFC feature extraction works best. After convert a signal into a freq domain, it's requered to convert it into a useable form. **Mel Frequency Cepstral Coefficients (MFCC)** is a good way to do that. *MFCC* takes the power spectrum of a signal and then uses a combination of filter banks and disrete cosinetransform to extract pattern of phones or features.

After extracting **MFFC** features we exract data into single data matrix, and a label vector with the correct label for eac data file is ceated. 

In [72]:
from scipy.io import wavfile

def get_mfcc(fpaths):
    data = []

    mfcc_max_length = 0

    # first file desitination name and index
    file_name = ''
    word_spoken_index = 0

    for n,file in enumerate(fpaths):

        # show current desintation 
        if (file.find(file_name)<=0):
            file_name=word_spoken[word_spoken_index]
            print(word_spoken[word_spoken_index])
            word_spoken_index+=1

        # read file 
        sampling_freq, audio = wavfile.read(file)
        # Extract MFCC features
        mfcc_features = mfcc(audio, sampling_freq)

        mfcc_len=mfcc_features.shape[0]*mfcc_features.shape[1]
        # get length of largets feature array
        if mfcc_len>mfcc_max_length:    
            mfcc_max_length=mfcc_len


        # flat data into 2D array
        mfcc_features=np.resize(mfcc_features,(1,mfcc_len))

        data.insert(n,mfcc_features)
        
    return data

In [73]:
data=get_mfcc(fpaths)
boy_data=get_mfcc(boy_fpaths)

0_zero
1_one
2_two
3_three
4_four
5_five
6_six
7_seven
8_eight
9_nine
aboard
adjusted & locked
All switches
Alternate air door
A_C Documents
Battery+Main bus
Cabin doors
Checked
Circuit breakers
Cockpit
Cockpit cheklist completed
Completed
Decimal
Flight Controls
Fuel Quantity
Fuel Selector
Fuel Shutoff Valve
Fuel Temperature
in
locked
off
On
open
preflight inspection
removed
Seats & Belts
Shut-off cabin heat
sufficient
Towbar
Weight and balance
0_zero
1_one
2_two
3_three
4_four
5_five
6_six
7_seven
8_eight
9_nine
aboard
adjusted & locked
All switches
Alternate air door
A_C Documents
Battery+Main bus
Cabin doors
Checked
Circuit breakers
Cockpit
Cockpit cheklist completed
Completed
Decimal
Flight Controls
Fuel Quantity
Fuel Selector
Fuel Shutoff Valve
Fuel Temperature
in
locked
off
On
open
preflight inspection
removed
Seats & Belts
Shut-off cabin heat
sufficient
Towbar
Weight and balance


#### Get Labels

In [74]:
def get_labels(labels, label_set):
    label_idx_dic={} # dict for storing labels

    #Each sample file is one row in data, and has one entry in labels
    print('Number of files total:', len(labels))
    all_labels = np.zeros(len(labels),dtype=int)
    for n, l in enumerate(label_set):
        label_idx_dic[n]=l
        all_labels[np.array([i for i, _ in enumerate(labels) if _ == l])] = n
    print('Labels and label indices', all_labels)
    
    return label_idx_dic, all_labels

In [75]:
_, all_label = get_labels(labels,label_set)
label_dic, boy_all_label = get_labels(boy_labels,boy_label_set)

Number of files total: 7108
Labels and label indices [ 0  0  0 ... 30 30 30]
Number of files total: 1969
Labels and label indices [ 0  0  0 ... 30 30 30]


In [80]:
# save label dictionary
def save_dict_to_file(dic,dataset):
    f = open('dict{}.txt'.format(dataset),'w')
    f.write(str(dic))
    f.close()

# load label dictionary
def load_dict_from_file(dataset):
    f = open('dict{}.txt'.format(dataset),'r')
    data=f.read()
    f.close()
    return eval(data)

In [82]:
# save dict
save_dict_to_file(label_dic,dataset)

# read dict
label_idx_dic = load_dict_from_file(dataset)

In [83]:
print(label_idx_dic)

{0: '0_zero', 1: '1_one', 2: '2_two', 3: '3_three', 4: '4_four', 5: '5_five', 6: '6_six', 7: '7_seven', 8: '8_eight', 9: '9_nine', 10: 'A_C Documents', 11: 'All switches', 12: 'Alternate air door', 13: 'Battery+Main bus', 14: 'Cabin doors', 15: 'Checked', 16: 'Circuit breakers', 17: 'Cockpit', 18: 'Cockpit cheklist completed', 19: 'Completed', 20: 'Decimal', 21: 'Flight Controls', 22: 'Fuel Quantity', 23: 'Fuel Selector', 24: 'Fuel Shutoff Valve', 25: 'Fuel Temperature', 26: 'On', 27: 'Seats & Belts', 28: 'Shut-off cabin heat', 29: 'Towbar', 30: 'Weight and balance', 31: 'aboard', 32: 'adjusted & locked', 33: 'in', 34: 'locked', 35: 'off', 36: 'open', 37: 'preflight inspection', 38: 'removed', 39: 'sufficient'}


#### Pad Zeros to Small Arrays

In [85]:
def data_x_y(data,all_labels):
    x_data = np.zeros((len(data),2808+1), dtype=float)

    for i,_d in enumerate(data):

        if (_d.shape[1]>2808):
            x_data[i,1:]=_d[0,0:2808]
        else :
            x_data[i,1:_d.shape[1]+1]=_d

        x_data[i,0]=all_labels[i]
        
    return x_data

In [87]:
# save all features data
np.savetxt('data{}.csv'.format('girl'), data_x_y(data,all_label), delimiter=',')
np.savetxt('data{}.csv'.format('boy'), data_x_y(boy_data,boy_all_label), delimiter=',')